In [1]:

from polymarket_analysis.analytics.gbm_param_estimator import GBMParameterEstimator


gbm = GBMParameterEstimator(symbol="BTC-USD", period="400d")
gbm.fetch_data()
gbm.calculate_returns()
gbm.estimate_parameters()

Fetched 400 price points for BTC-USD
Calculated 399 daily returns


{'mu_daily': np.float64(0.001491057218510357),
 'sigma_daily': np.float64(0.025244149411078137),
 'mu_annualized': np.float64(0.5442358847562803),
 'sigma_annualized': np.float64(0.4822887973127982)}

In [1]:
from polymarket_analysis.data.load_history import load_price_history
from polymarket_analysis.data.markets import load_raw_markets


raw_markets = load_raw_markets()

print(f"Loaded {len(raw_markets)} markets.")

Loaded 9819 markets.


In [8]:
import pandas as pd
markets = pd.DataFrame(raw_markets)

In [3]:
import importlib
from polymarket_analysis.data import load_history
importlib.reload(load_history)

<module 'polymarket_analysis.data.load_history' from '/Users/vobornij/projects/polymarket/src/polymarket_analysis/data/load_history.py'>

In [4]:
from polymarket_analysis.data.contract_loader import ContractLoader

contracts = ContractLoader().load_crypto_contracts(raw_markets)

TypeError: list indices must be integers or slices, not str

In [ ]:
token_histories = [load_price_history(contract) for contract in contracts[300:310]]
nonempty_histories = [history for history in token_histories if history is not None]
token_history = nonempty_histories[0]

No price history found for contract Will the price of Bitcoin be greater than $112K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be between $108K and $110K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be between $110K and $112K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be between $106K and $108K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be between $110K and $112K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be between $106K and $108K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be less than $106K on June 17? - Yes
No price history found for contract Will the price of Bitcoin be less than $106K on June 17? - Yes


In [ ]:
from polymarket_analysis.data.binance import Binance

polymarket_start = token_history.price_df['timestamp'].min()
polymarket_end = token_history.contract.end_date

btc_df = Binance.load_bitcon_5min(
    from_date=polymarket_start, 
    to_date=polymarket_end
)
print(f"Filtered Bitcoin data between {polymarket_start}, {polymarket_end}: {len(btc_df)} rows")

Filtered Bitcoin data between 2025-06-10 18:20:07+00:00, 2025-06-17 00:00:00+00:00: 1796 rows


In [ ]:
# Calculate probability of BTC above 103k at each token's end date
from datetime import datetime, timezone
import pandas as pd

def calculate_prob_above_target(row, target_price):
    end_date = token_history.contract.target_time
    
    # Get current date (or the date when prediction is made)
    # Using the date from the row or current date
    current_date = pd.to_datetime(row['timestamp'])
    
    # Calculate days ahead (fractional days for precision)
    time_diff = end_date - current_date
    days_ahead = time_diff.total_seconds() / (24 * 3600)

    return gbm.p_above_target(row['price'], target_price, days_ahead)
    

# Apply the function to calculate probabilities
btc_df['estimated_prob_above_103k'] = btc_df.apply(
    lambda row: calculate_prob_above_target(row, target_price=103000),
    axis=1
)

# Display results
print("Sample of estimated probabilities:")
print(btc_df[['timestamp', 'estimated_prob_above_103k']].head(10))

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects.

In [ ]:
# %matplotlib tk
%matplotlib inline

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

for mh in [token_history]:
    fig.add_trace(go.Scatter(
            x=mh.price_df['timestamp'], 
            y=mh.price_df['price'],
            mode='lines+markers',
            name=f"{mh.slug}",
        ))

# Add Bitcoin price
fig.add_trace(
    go.Scatter(
        x=btc_df['timestamp'], 
        y=btc_df['price'],
        mode='lines',
        name='Bitcoin Price',
        # line=dict(color='black'),
    ),
    secondary_y=True,
)

# Add Bitcoin estimate
fig.add_trace(
    go.Scatter(
        x=btc_df['timestamp'], 
        y=btc_df['estimated_prob_above_103k'],
        mode='lines',
        name='Bitcoin Estimate (P(BTC > 103k))',
        # line=dict(color='black'),
    ),
)

# Update layout
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Polymarket Price", secondary_y=False)
fig.update_yaxes(title_text="Bitcoin Price (USD)", secondary_y=True)

fig.update_layout(
    title_text="Polymarket vs Bitcoin Price Comparison (5-minute data)",
    legend=dict(
        orientation="h",  # horizontal orientation
        yanchor="top",
        y=-0.1,          # position below the plot
        xanchor="center",
        x=0.5
    ),
    showlegend=True
)

fig.show(renderer="browser")